## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Detect Cancer Genetics](https://aws.amazon.com/marketplace/pp/prodview-ju5im2hqfk4bo)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Detect Cancer Genetics


- **Model**: `en.med_ner.bionlp.pipeline`
- **Model Description**: Pretrained named entity recognition pipeline for biology and genetics terms.
- **Predicted Entities:** `Amino_acid`, `Anatomical_system`, `Cancer`, `Cell`, `Cellular_component`, `Developing_anatomical_structure`, `Gene_or_gene_product`, `Immaterial_anatomical_entity`, `Multi-tissue_structure`, `Organ`, `Organism`, `Organism_subdivision`, `Organism_substance`, `Pathological_formation`, `Simple_chemical`, `Tissue`

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "en-med-ner-bionlp-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)


### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

[01/06/25 11:39:06] INFO     Creating model with name:                                              ]8;id=808352;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=150045;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             en-med-ner-bionlp-pipeline-2025-01-06-11-39-05-987                                    

                    INFO     Creating endpoint-config with name en-med-ner-bionlp-pipeline          ]8;id=866587;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=999660;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name en-med-ner-bionlp-pipeline                 ]8;id=11706;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=587782;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

-------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    '''Under loupe magnification, the lesion was excised with 2 mm margins, oriented with sutures and submitted for frozen section pathology. The report was "basal cell carcinoma with all margins free of tumor." Hemostasis was controlled with the Bovie. Excised lesion diameter was 1.2 cm. The defect was closed by elevating a left laterally based rotation flap utilizing the glabellar skin. The flap was elevated with a scalpel and Bovie, rotated into the defect without tension, sutured to the defect with scissors and inset in layers with interrupted 5-0 Vicryl for the dermis and running 5-0 Prolene for the skin. Donor site was closed in V-Y fashion with similar suture technique.''',
    '''The patient was admitted for symptoms that sounded like postictal state. He was initially taken to the hospital. CT showed edema and slight midline shift, and therefore he was transferred here. He has been seen by the Hospitalists Service. He has not experienced a recurrent seizure. Electroencephalogram shows slowing. MRI of the brain shows a large inhomogeneous infiltrating right frontotemporal neoplasm surrounding the right middle cerebral artery. There is inhomogeneous uptake consistent with possible necrosis. He also has had a SPECT image of his brain, consistent with neoplasm, suggesting a relatively high-grade neoplasm. The patient was diagnosed with a brain tumor in 1999. We do not yet have all the details. He underwent a biopsy by Dr. Y. One of the notes suggested that this was a glioma, likely an oligodendroglioma, pending a second opinion at Clinic. That is not available on the chart as I dictate.''',
]


sample_text = '''The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomic organization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene for Type II diabetes mellitus in the Pima Indian population. The gene spans approximately 7.6 kb and contains one noncoding and two coding exons separated by approximately 2.2 and approximately 2.6 kb introns, respectively. We identified 14 single nucleotide polymorphisms (SNPs), including one that predicts a Val366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Our expression studies revealed the presence of the transcript in various human tissues including pancreas, and two major insulin-responsive tissues: fat and skeletal muscle. The characterization of the KCNJ9 gene should facilitate further studies on the function of the KCNJ9 protein and allow evaluation of the potential role of the locus in Type II diabetes.'''

### JSON

In [9]:
input_json_data = {"text": sample_text}
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"][0])

,ner_chunk,begin,end,ner_label,ner_confidence
0,human,4,8,Organism,0.9996
1,Kir 3.3,17,23,Gene_or_gene_product,0.99635
2,GIRK3,26,30,Gene_or_gene_product,1.0
3,potassium,92,100,Simple_chemical,0.9452
4,GIRK,103,106,Gene_or_gene_product,0.998
5,chromosome 1q21-23,189,206,Cellular_component,0.80295
6,Type II,232,238,Gene_or_gene_product,0.9992
7,human,681,685,Organism,1.0
8,tissues,687,693,Tissue,0.9999
9,pancreas,705,712,Organ,0.9995


### JSON Lines

In [10]:
import json

def create_jsonl(records):

    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "human", "begin": 4, "end": 8, "ner_label": "Organism", "ner_confidence": "0.9996"}, {"ner_chunk": "Kir 3.3", "begin": 17, "end": 23, "ner_label": "Gene_or_gene_product", "ner_confidence": "0.99635"}, {"ner_chunk": "GIRK3", "begin": 26, "end": 30, "ner_label": "Gene_or_gene_product", "ner_confidence": "1.0"}, {"ner_chunk": "potassium", "begin": 92, "end": 100, "ner_label": "Simple_chemical", "ner_confidence": "0.9452"}, {"ner_chunk": "GIRK", "begin": 103, "end": 106, "ner_label": "Gene_or_gene_product", "ner_confidence": "0.998"}, {"ner_chunk": "chromosome 1q21-23", "begin": 189, "end": 206, "ner_label": "Cellular_component", "ner_confidence": "0.80295"}, {"ner_chunk": "Type II", "begin": 232, "end": 238, "ner_label": "Gene_or_gene_product", "ner_confidence": "0.9992"}, {"ner_chunk": "human", "begin": 681, "end": 685, "ner_label": "Organism", "ner_confidence": "1.0"}, {"ner_chunk": "tissues", "begin": 687, "end": 693, "ner_label": "Tissue", "ner_confidenc

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

[01/06/25 11:43:10] INFO     Deleting endpoint with name: en-med-ner-bionlp-pipeline                ]8;id=331457;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=105553;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\

[01/06/25 11:43:11] INFO     Deleting endpoint configuration with name: en-med-ner-bionlp-pipeline  ]8;id=186957;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=562410;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\

## 4. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

{'predictions': [[{'ner_chunk': 'lesion',
    'begin': 31,
    'end': 36,
    'ner_label': 'Pathological_formation',
    'ner_confidence': '1.0'},
   {'ner_chunk': 'basal cell carcinoma',
    'begin': 151,
    'end': 170,
    'ner_label': 'Cancer',
    'ner_confidence': '0.9991333'},
   {'ner_chunk': 'tumor',
    'begin': 197,
    'end': 201,
    'ner_label': 'Cancer',
    'ner_confidence': '0.9999'},
   {'ner_chunk': 'lesion',
    'begin': 255,
    'end': 260,
    'ner_label': 'Pathological_formation',
    'ner_confidence': '0.5082'},
   {'ner_chunk': 'glabellar skin',
    'begin': 369,
    'end': 382,
    'ner_label': 'Organ',
    'ner_confidence': '0.79815'},
   {'ner_chunk': 'flap',
    'begin': 389,
    'end': 392,
    'ner_label': 'Tissue',
    'ner_confidence': '0.9869'},
   {'ner_chunk': 'dermis',
    'begin': 566,
    'end': 571,
    'ner_label': 'Tissue',
    'ner_confidence': '0.9969'},
   {'ner_chunk': 'skin',
    'begin': 605,
    'end': 608,
    'ner_label': 'Organ',
    

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "lesion", "begin": 31, "end": 36, "ner_label": "Pathological_formation", "ner_confidence": "1.0"}, {"ner_chunk": "basal cell carcinoma", "begin": 151, "end": 170, "ner_label": "Cancer", "ner_confidence": "0.9991333"}, {"ner_chunk": "tumor", "begin": 197, "end": 201, "ner_label": "Cancer", "ner_confidence": "0.9999"}, {"ner_chunk": "lesion", "begin": 255, "end": 260, "ner_label": "Pathological_formation", "ner_confidence": "0.5082"}, {"ner_chunk": "glabellar skin", "begin": 369, "end": 382, "ner_label": "Organ", "ner_confidence": "0.79815"}, {"ner_chunk": "flap", "begin": 389, "end": 392, "ner_label": "Tissue", "ner_confidence": "0.9869"}, {"ner_chunk": "dermis", "begin": 566, "end": 571, "ner_label": "Tissue", "ner_confidence": "0.9969"}, {"ner_chunk": "skin", "begin": 605, "end": 608, "ner_label": "Organ", "ner_confidence": "0.9997"}]}
{"predictions": [{"ner_chunk": "patient", "begin": 4, "end": 10, "ner_label": "Organism", "ner_confidence": "1.0"}, {"ne

In [21]:
model.delete_model()

[01/06/25 11:56:25] INFO     Deleting model with name:                                              ]8;id=550378;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=54592;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5226\5226]8;;\
                             en-med-ner-bionlp-pipeline-2025-01-06-11-50-16-248                                    

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

